In [ ]:
# Set up data & files
!mkdir data
!wget -O data/iris.csv https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/iris/data/iris.csv
!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/iris/iris_utils_native_keras.py

In [ ]:
!pip install -U -q tfx-bsl==0.21.1
!sudo pip3 install -U -q --trusted-host pypi.org google protobuf3 # protobuf3
# !pip install -U -q tfx==0.21.0
# !sudo apt install protobuf-compiler
!git clone https://github.com/tensorflow/tfx /content/tfx-source && pushd /content/tfx-source
# !cd tfx-source && pip install -e.
!cd tfx-source/ && python setup.py

In [ ]:
# import google.protobuf.Any
!head tfx/proto/example_gen.proto


In [ ]:
!cd tfx-source/ && ls && python setup.py egg_info
# !pip freeze | grep proto

In [ ]:
!find / -name any.proto

In [ ]:
# !python -V
# !touch /usr/local/lib/python3.6/dist-packages/google/__init__.py
!ls /usr/local/lib/python3.6/dist-packages/google/



In [ ]:
!ls /usr/local/lib/python2.7/site-packages/WHEEL

In [1]:
import os
from typing import Text

import absl
import tensorflow_model_analysis as tfma

from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
# from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input

from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext

/Users/i854694/tfx-source/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [2]:
context = InteractiveContext()
module_file = os.path.abspath("iris_utils_native_keras.py")

In [3]:
examples = external_input('data/')

# Brings data into the pipeline or otherwise joins/converts training data.
example_gen = CsvExampleGen(input=examples)
context.run(example_gen)

ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.example_gen.csv_example_gen.component.CsvExampleGen


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/CsvExampleGen/examples/1, id: 2)]
        ))

In [4]:
# statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
# context.run(statistics_gen)

statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-21T09_43_05.738085"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-21T09_43_05.738085"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-21T09_43_05.738085');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.statistics_gen.component.StatisticsGen
/Users/i854694/.virtualenvs/pipelines/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:236: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/StatisticsGen/statistics/2, id: 3)]
        ))

In [5]:
infer_schema = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(infer_schema)

context.show(infer_schema.outputs['schema'])

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-21T09_43_05.738085"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-21T09_43_05.738085"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-21T09_43_05.738085');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.schema_gen.component.SchemaGen


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


,Type,Presence,Valency,Domain
Feature name,,,,
'petal_length',FLOAT,required,,-
'petal_width',FLOAT,required,,-
'sepal_length',FLOAT,required,,-
'sepal_width',FLOAT,required,,-
'variety',INT,required,,-


In [6]:
validate_stats = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=infer_schema.outputs['schema'])
context.run(validate_stats)

context.show(validate_stats.outputs['anomalies'])

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-21T09_43_05.738085"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-21T09_43_05.738085"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-21T09_43_05.738085');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.example_validator.component.ExampleValidator


In [7]:
transform = Transform(
      examples=example_gen.outputs['examples'],
      schema=infer_schema.outputs['schema'],
      module_file=module_file)

context.run(transform)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-21T09_43_05.738085"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-21T09_43_05.738085"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-21T09_43_05.738085');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.transform.component.Transform


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/Transform/transform_graph/5/.temp_path/tftransform_tmp/7aaaa7b64d1e48d8b436100801c87e27/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning a

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/Transform/transformed_examples/5, id: 7)]
        ))

In [9]:
trainer = Trainer(
    module_file=module_file,
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=infer_schema.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))

context.run(trainer)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-21T09_43_05.738085"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-21T09_43_05.738085"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-21T09_43_05.738085');


Train for 10000 steps, validate for 5000 steps
10000/10000 [==============================] - 33s 3ms/step - loss: 0.0191 - accuracy: 0.9958 - val_loss: 0.9094 - val_accuracy: 0.9459
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/Trainer/model/7/serving_model_dir/assets
Train for 10000 steps, validate for 5000 steps
10000/10000 [==============================] - 31s 3ms/step - loss: 0.0273 - accuracy: 0.9930 - val_loss: 0.4818 - val_accuracy: 0.9459
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets written to: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/Trainer/model/7/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/Trainer/model/7, id: 9)]
        ))

In [10]:
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='variety')],
        slicing_specs=[tfma.SlicingSpec(feature_keys=['sepal_length'])]))
  
context.run(model_analyzer)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-21T09_43_05.738085"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-21T09_43_05.738085"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-21T09_43_05.738085');
ERROR:absl:mlmd client NotFoundError: No type found for query: tfx.components.evaluator.component.Evaluator


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/Evaluator/evaluation/8, id: 10)]
        ))

In [15]:
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='variety')], 
        metrics_specs=[
                              tfma.MetricsSpec(metrics=[
                                  tfma.config.MetricConfig(
                                      class_name='ExampleCount',
                                      # Count > 0, OK.
                                      threshold=tfma.config.MetricThreshold(
                                          value_threshold=tfma
                                          .GenericValueThreshold(
                                              lower_bound={'value': 0}))),])]#,
        #slicing_specs=[tfma.SlicingSpec(feature_keys=['sepal_length'])]
    ))
  
context.run(model_analyzer)

ERROR:absl:mlmd client AlreadyExistsError: Given node already exists: type_id: 2
name: "interactive-2020-02-21T09_43_05.738085"
properties {
  key: "pipeline_name"
  value {
    string_value: "interactive-2020-02-21T09_43_05.738085"
  }
}
Internal: Error when executing query: UNIQUE constraint failed: Context.type_id, Context.name query:  INSERT INTO `Context`(    `type_id`, `name` ) VALUES(2, 'interactive-2020-02-21T09_43_05.738085');


ExecutionResult(
    component_id: Evaluator
    execution_id: 10
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: /var/folders/jy/nxgywhx174z8xwy8h6d6mls00000gn/T/tfx-interactive-2020-02-21T09_43_05.738085-zk7tf2nz/Evaluator/evaluation/10, id: 12)]
        ))

In [17]:
context.show(model_analyzer.outputs['evaluation'])